In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session





In [ ]:
file = "/kaggle/input/health-care-data-set-on-heart-attack-possibility/heart.csv"
df = pd.read_csv(file)
df.head()

Attribute Information<br>
1) age<br>
2) sex<br>
3)  cp: chest pain type<br>
-- Value 1: typical angina<br>
-- Value 2: atypical angina<br>
-- Value 3: non-anginal pain<br>
-- Value 4: asymptomatic<br>
4) trestbps: resting blood pressure (in mm Hg on admission to the hospital)<br>
5) chol: serum cholestoral in mg/dl <br>
6) fasting blood sugar > 120 mg/dl<br>
7) resting electrocardiographic results (values 0,1,2)<br>
8) thalach: maximum heart rate achieved<br>
9) exercise induced angina<br>
10) oldpeak = ST depression induced by exercise relative to rest<br>
11) the slope of the peak exercise ST segment<br>
12) number of major vessels (0-3) colored by flourosopy<br>
13) thal: 0 = normal; 1 = fixed defect; 2 = reversable defect<br>
14) target: 0= less chance of heart attack 1= more chance of heart attack<br>

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
plt.rcParams['figure.figsize'] = [15, 5]

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [ ]:
X["cp"].value_counts()

In [ ]:
y.value_counts()

In [ ]:
fig = px.bar(x=y.unique(), y=y.value_counts(), title="Clases", labels={'y':"Frecuencia"})
fig.update_xaxes(title="Values")
fig.update_layout(
xaxis = dict(
    tickmode = "array",
    tickvals = [0,1],
    ticktext = ["NO", "YES"]
))
fig.show()

In [ ]:
X.hist(figsize=(16,20));

In [ ]:
df.groupby("target").mean()

In [ ]:
table_sex = pd.crosstab(df.sex, df.target)
table_sex
table_sex.div(table_sex.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title="Sex - target",ylabel="counts").set_xticklabels(["Female", "Male"]);

In [ ]:
sc = StandardScaler()
X_sc = sc.fit_transform(X)
X_sc[:,:5]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.3, random_state=420)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print("Accuracy on Test Set: {:.2f}".format(logreg.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(logreg, X_test, y_test, display_labels = ["0","1"], cmap=plt.cm.Blues, normalize="true")
plt.title("Confusion Matrix");

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
#binary roc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

The higher the AUC, the better model is at distinguishing between patients with the desease and no desease. Capability of the model to predict Class 0 as Class 0, and Class 1 as Class 1. 1 means Excelent and 0 is no separability.
ROC is a distribution curve. The distribution of each probability should have 0 overlap meaning an indeal measure of separability.
Sensivity and Specificity are inversely proportional to each other. When Sensitivity increase, Specificity decreases.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

scores = cross_val_score(logreg, X_sc, y, cv=10)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
# skf = StratifiedKFold(n_splits=10, random_state=420,shuffle=True)

In [ ]:
# print(logreg.predict([f]))
# print(logreg.predict_proba([f]))

In [ ]:
!pip install shap
import shap;

In [ ]:
print(X.columns)
explainer = shap.Explainer(logreg, X)
shap_values = explainer(X)
shap.plots.beeswarm(shap_values, color = plt.get_cmap("cool"))

Top features that impact the model’s output are displayed on the top. The x position of the dot is determined by the SHAP value of that feature. Thalac, chol, trestbps, age, cp are the have the most prominent value.

In [ ]:
table_sex = pd.crosstab(df.thalach, df.target)
table_sex
table_sex.div(table_sex.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title="thalach - target",ylabel="counts");

In [ ]:
table_sex = pd.crosstab(df.trestbps, df.target)
table_sex
table_sex.div(table_sex.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title="trestbps - target",ylabel="counts")

In [ ]:
table_sex = pd.crosstab(df.chol, df.target)
table_sex
table_sex.div(table_sex.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title="chol - target",ylabel="counts");

In [ ]:
table_sex = pd.crosstab(df.age, df.target)
table_sex
table_sex.div(table_sex.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title="Age - target",ylabel="counts");